In [1]:
from transformers import RobertaTokenizer
from transformers import RobertaTokenizer, RobertaModel
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from IPython.display import clear_output

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import time
import math

from sklearn.preprocessing import normalize
import pandas as pd
from tqdm import tqdm
from torch import nn
import random as rnd
import torch
import ast
import numpy as np
import re

In [2]:

def extract_sentences():

    # Extract src data
    source_data = []
    top_data = []
    top_decoupled_data = []

    # Read source.txt
    with open('../dataset/source.txt', 'r') as file:
        source_data = ast.literal_eval(file.read())

    # Read labels.txt
    with open("../dataset/input_labels.txt", 'r') as file:
        labels = file.read().splitlines()

    return source_data, labels

In [3]:
def read_unique_labels(file_path):
    labels = []
    with open(file_path, 'r') as file:
        labels = file.read().splitlines()
    return labels

In [4]:
def read_dev_dataset(file_path):

    with open(file_path, 'r') as file:
        text = file.read()
        
    pattern_src = r'(?<="dev\.SRC": ").+(?=", "dev\.EXR")'
    pattern_top = r'(?<="dev\.TOP": ").+(?=", "dev\.PCFG_ERR")'
    test_src = re.finditer(pattern_src, text)
    test_top = re.finditer(pattern_top, text)
    test_src_arr = []
    test_top_decoupled_arr = []

    for match in test_src:
        test_src_arr.append(match.group())
    
    pattern_top_decoupled = r'(?<=\))[\w ]*(?= \()|(?<=ORDER)[\w ]*(?= \()|(?<=PIZZAORDER)[\w ]*(?= \()|(?<=DRINKORDER)[\w ]*(?= \()'
    for match in test_top:
        temp = re.sub(pattern_top_decoupled,'',match.group())
        test_top_decoupled_arr.append(temp)

    return test_src_arr, test_top_decoupled_arr

#read_dev_dataset("../dataset/PIZZA_dev.json")

In [5]:
def parse_tc(train_SRC,train_TOP):
    
    def parse_sexp(s):
        s = s.replace('(', ' ( ').replace(')', ' ) ')
        tokens = s.split()
        def helper(tokens):
            token = tokens.pop(0)
            if token == '(':
                L = []
                while tokens[0] != ')':
                    L.append(helper(tokens))
                tokens.pop(0)
                return L
            else:
                return token
        return helper(tokens.copy())

    tree = parse_sexp(train_TOP)

    entities = []

    def extract_entities(tree, current_label=None, text_accumulator=[]):
        if isinstance(tree, list):
            label = tree[0]
            content = tree[1:]
            text = []
            for item in content:
                extract_entities(item, label, text)
            entity_text = ' '.join(text)
            #if label in ['ORDER', 'PIZZAORDER', 'NOT'] or label not in ['NUMBER']:
            match = re.search(re.escape(entity_text), train_SRC)
            if match:
                if label == "NOT":
                    temp_entity = entities.pop()
                    entities.append({
                    'label': label+"-"+temp_entity['label'],
                    'word': match.group(),
                    })

                else:
                    entities.append({
                        'label': label,
                        'word': match.group(),
                    })
            text_accumulator.extend(text)
        else:
            text_accumulator.append(tree)

    extract_entities(tree)

    result = {
        'sentence': train_SRC,
        'entities': entities
    }
    #print(result)
    return result

def generate_bio_tags(sentence, entities):
    
    words = sentence.split()
    bio_tags = ["0"] * len(words)  
    
    
    for entity in entities:
        label = entity['label'] 
        entity_words = entity['word'].split()  
        
        
        if label in ['PIZZAORDER', 'ORDER', 'DRINKORDER','COMPLEX_TOPPING']:
            continue
        
        for i in range(len(words)):
            if words[i:i+len(entity_words)] == entity_words:
                bio_tags[i] = f"B-{label}"  
                for j in range(1, len(entity_words)):
                    bio_tags[i+j] = f"I-{label}"  
                break  
    
    return list(zip(words, bio_tags))

In [6]:
def create_test_labels_input():
    longest_sentence = 0
    unique_words = set()
    result = []
    tags = []
    ut_labels = read_unique_labels('./unique_labels.txt')
    t_labels = {}
    t_labels['0'] = 0
    for i in range(len(ut_labels)):
        t_labels[ut_labels[i]] = i+1
    
    test_SRC, test_TOP_DECOUPLED = read_dev_dataset("../dataset/PIZZA_dev.json")
    test_SRC_size = len(test_SRC)
    print(test_SRC[0])
    print(len(test_SRC))
    print(test_TOP_DECOUPLED[0])
    print(len(test_TOP_DECOUPLED))

    with open('../dataset/test_input_labels.txt', 'w') as f:
        for i in range(test_SRC_size):
            test_SRC_item = test_SRC[i]
            test_TOP_DECOUPLED_item = test_TOP_DECOUPLED[i]
            longest_sentence = max(len(test_SRC_item.split()), longest_sentence)    
            unique_words.update(test_SRC_item.split())            

            result.append(parse_tc(test_SRC_item,test_TOP_DECOUPLED_item))

            tags.append(generate_bio_tags(result[i]['sentence'], result[i]['entities']))

            test_SRC_labels_list = []
            for word, tag in tags[i]:
                test_SRC_labels_list.append(tag)
                f.write(f"{tag} ")
            f.write("\n")
    print("longest sentence:",longest_sentence)

create_test_labels_input()


i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage
348
(ORDER (PIZZAORDER (NUMBER two ) (SIZE medium ) (TOPPING sausage ) (TOPPING black olives ) ) (PIZZAORDER (NUMBER two ) (SIZE medium ) (TOPPING pepperoni ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) (PIZZAORDER (NUMBER three ) (SIZE large ) (TOPPING pepperoni ) (TOPPING sausage ) ) )
348
longest sentence: 33


In [7]:
def extract_word2vec_embeddings(data, device='cuda'):
    """
    Extract Word2Vec embeddings
    """
    sentences = [sentence.split() for sentence in data]
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    
    # Convert to GPU tensor if possible
    embeddings = []
    for sentence in sentences:
        # Get word vectors, use zeros if word not in vocabulary
        sent_embedding = [model.wv[word] for word in sentence if word in model.wv] or [np.zeros(100)]
        embedding = torch.tensor(np.mean(sent_embedding, axis=0), device=device)
        embeddings.append(embedding)
    
    return torch.stack(embeddings)

def extract_contextual_embeddings(data, device='cuda'):
    """
    Extract RoBERTa contextual embeddings
    """
    # Move model to GPU if available
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaModel.from_pretrained('roberta-base').to(device)
    
    embeddings = []
    for sentence in data:
        # Tokenize and move to GPU
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Mean pooling and move back to CPU if needed
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
        embeddings.append(embedding.cpu().numpy())
    
    return np.array(embeddings)

def extract_lda_features(data, n_topics=2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(data)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda_features = lda.fit_transform(tfidf_matrix)
    return lda_features

In [8]:
def lstm_embedding(data):
    word2vec_embeddings = extract_word2vec_embeddings(data)
    #print(word2vec_embeddings)
    print(word2vec_embeddings.shape)
    # lda_features = extract_lda_features(data)
    # contextual_embeddings = extract_contextual_embeddings(data)

    # word2vec_embeddings = word2vec_embeddings.cpu().numpy()
    # combined_features = np.hstack((word2vec_embeddings, lda_features, contextual_embeddings))
    return word2vec_embeddings

class NERDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, max_len):
    """
    This is the constructor of the NERDataset
    Inputs:
    - x: a list of lists where each list contains the ids of the tokens
    - y: a list of lists where each list contains the label of each token in the sentence
    - pad: the id of the <PAD> token (to be used for padding all sentences and labels to have the same length)
    """
    # i guess x should be extended to have the same length as y
    self.x_tensor = [seq + "<pad>" * (max_len - len(seq)) for seq in x]
    self.y_tensor = torch.tensor([seq + [0] * (max_len - len(seq)) for seq in y], dtype=torch.long)
    #################################################################################################################

  def __len__(self):
    """
    This function should return the length of the dataset (the number of sentences)
    """
    ###################### TODO: return the length of the dataset #############################

    return len(self.x_tensor)
  
    ###########################################################################################

  def __getitem__(self, idx):
    """
    This function returns a subset of the whole dataset
    """
    ###################### TODO: return a tuple of x and y ###################################
    return self.x_tensor[idx], self.y_tensor[idx]
    ##########################################################################################

def save_model(model, epoch, path="model_epoch_{}.pth"):
    torch.save(model.state_dict(), path.format(epoch))
    print(f"Model saved to {path.format(epoch)}")

In [9]:
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours}h {minutes}m {seconds}s"

In [10]:
class SimulatedAnnealingLRScheduler:
    def __init__(self, 
                 initial_lr=0.01, 
                 min_lr=1e-5, 
                 max_lr=1e-3, 
                 batch_num=100, 
                 temperature_init=1.0, 
                 cooling_rate=0.95):
        """
        Simulated Annealing Learning Rate Scheduler
        
        Parameters:
        - initial_lr: Starting learning rate
        - min_lr: Minimum learning rate
        - max_lr: Maximum learning rate
        - total_epochs: Total training epochs
        - temperature_init: Initial temperature
        - cooling_rate: Rate of temperature decrease
        """
        self.initial_lr = initial_lr
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.batch_num = batch_num
        self.temperature_init = temperature_init
        self.cooling_rate = cooling_rate
        
        # Dynamic learning rate tracking
        self.current_lr = initial_lr
    
    def get_lr(self, batch_num):
        """
        Compute learning rate based on simulated annealing
        
        Parameters:
        - batch_num: Current batch number
        
        Returns:
        - Adjusted learning rate
        """
        # Calculate current temperature
        current_temp = self.temperature_init * (self.cooling_rate ** batch_num)
        
        # Simulated annealing learning rate calculation
        # Smoothly transitions from exploration to exploitation
        new_lr = (
            self.max_lr * (1-math.exp(-current_temp)) + 
            self.min_lr * (math.exp(-current_temp))
        )
        
        # Ensure learning rate stays within bounds
        new_lr = max(self.min_lr, min(new_lr, self.max_lr))
        
        self.current_lr = new_lr
        return new_lr


In [11]:
#LSTM model
class NER(nn.Module):
    def __init__(self, output_dim, hidden_size=512, embedding_dim=768,num_layers=1,dropout=0.1,bidirectional=True,batch_first=True,use_attention=False):
        """
        The constructor of our NER model
        Inputs:
        - vacab_size: the number of unique words
        - embedding_dim: the embedding dimension
        - output_dim: output dimension
        - hidden_size: the hidden size of the LSTM layer
        - num_layers: the number of LSTM layers
        - dropout: the dropout rate
        - bidirectional: whether to use bidirectional LSTM
        """
        super(NER, self).__init__()
        
        ## Word embedding layer        
        if bidirectional:
            hidden_size = hidden_size // 2
        # LSTM layer with combined embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_size,num_layers=num_layers, 
                            batch_first=batch_first, dropout=dropout, bidirectional=bidirectional)
        self.use_attention = use_attention
        if use_attention:
            if bidirectional:
                hidden_size *= 2
            self.attention = nn.ReLU()
        # Linear layer
        self.output_layer = nn.Linear(hidden_size, output_dim)

    def forward(self, embeddings):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        # LSTM and linear layers
        if embeddings.dim() == 2:
            embeddings = embeddings.unsqueeze(1)
        lstm_out, _ = self.lstm(embeddings)
        # Apply attention if needed

        if self.use_attention:
            # Compute attention weights
            attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
            # Apply weighted sum
            context = torch.sum(lstm_out * attention_weights, dim=1)
        else:
            # Use the last output if no attention
            context = lstm_out[:, -1, :]

        final_output = self.output_layer(context)
        
        return final_output
  
def train(model, train_dataset,norm_clip = 0.1, batch_size=512, epochs=5, learning_rate=0.00005 , save_path="models_lstm/model_epoch_{}.pth", log_file="logs_lstm.txt",load_pretrained=False):
    """
    This function implements the training logic
    Inputs:
    - model: the model ot be trained
    - train_dataset: the training set of type NERDataset
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    ############################## TODO: replace the Nones in the following code ##################################
    
    # (1) create the dataloader of the training set (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # (2) make the criterion cross entropy loss
    criterion = nn.CrossEntropyLoss()

    sa_scheduler = SimulatedAnnealingLRScheduler(
        initial_lr=1, 
        min_lr=5e-6, 
        max_lr=0.5, 
        batch_num=len(train_dataloader) * epochs,
    )
    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(), lr=sa_scheduler.initial_lr)
    epoch_times = []
    batch_times = []

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    else:
        print("CUDA is not available. Training on CPU ...")
    
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        start_time = time.time()
        batches_prev = epoch_num * len(train_dataloader)
        batch_idx = 0
        for train_input, train_label in tqdm(train_dataloader):
            current_lr = sa_scheduler.get_lr(batches_prev + batch_idx/len(train_dataloader))
        
                # Update optimizer learning rate
            for param_group in optimizer.param_groups:
                    param_group['lr'] = current_lr
            batch_start_time = time.time()

            # (4) move the train input to the device
            #embeddings_tensor = embeddings_tensor.to(device)
            
            embeddings = lstm_embedding(train_input)
            embeddings_tensor = embeddings.to(device)
            #embeddings_tensor = torch.tensor(embeddings).float()
            train_label = train_label.float()
            train_label = train_label.to(device)
            # (5) move the train label to the device

            # (6) do the forward pass
            output = model(embeddings_tensor)
            # output = output.permute(0, 2, 1) 

            batch_loss = criterion(output.reshape(-1), train_label.view(-1))

            # (8) append the batch loss to the total_loss_train
            total_loss_train += batch_loss.item()
            
            # (9) calculate the batch accuracy (just add the number of correct predictions)

            train_label = train_label.permute(1, 0) 
            acc = torch.sum(torch.argmax(output, dim=-1) == train_label)
            total_acc_train += acc

            # (10) zero your gradients
            optimizer.zero_grad()

            # (11) do the backward pass
            batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), norm_clip) 
            # (12) update the weights with your optimizer
            optimizer.step()
            # Calculate batch time
            clear_output(wait=True)

            print(
            f'Epochs: {epoch_num + 1} | Train Loss: {batch_loss.item()} \
            | Train Accuracy: {acc/(train_dataset.y_tensor.size(1)*batch_size)}\n')
            print(f"Current Learning Rate: {current_lr:.6f}")

            batch_time = time.time() - batch_start_time
            batch_times.append(batch_time)

            # Calculate remaining time
            avg_batch_time = sum(batch_times) / len(batch_times)
            remaining_batches = (epochs - epoch_num - 1) * len(train_dataloader) + (len(train_dataloader) - batch_idx - 1)
            remaining_time = avg_batch_time * remaining_batches

            # Print batch metrics and remaining time
            print(f"Epoch {epoch_num+1}/{epochs}, Batch {batch_idx+1}/{len(train_dataloader)}")#, Loss: {loss.item()}, Accuracy: {(predicted == train_label).sum().item() / train_label.numel()}")
            print(f"Time for batch {batch_idx+1}: {batch_time:.2f} seconds")
            print(f"Estimated remaining time: {format_time(remaining_time)} seconds")
            batch_idx += 1
        ##############################################################################################################    
        # epoch loss
        epoch_loss = total_loss_train / len(train_dataset)
        epoch_acc = total_acc_train / (len(train_dataset) * train_dataset.y_tensor.size(1))
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')
        print(f"Current Learning Rate: {current_lr:.6f}")

        # (13) calculate the accuracy
        # Calculate epoch time
        epoch_time = time.time() - start_time
        epoch_times.append(epoch_time)

        # Calculate remaining time
        avg_epoch_time = sum(epoch_times) / len(epoch_times)
        remaining_time = avg_epoch_time * (epochs - (epoch_num + 1))

        # Print epoch metrics and remaining time
        print(f"Epoch {epoch_num+1}/{epochs}, Loss: {total_loss_train/len(train_dataloader)}")#, Accuracy: {total_acc_train/len(train_dataloader.dataset)}")
        print(f"Time for epoch {epoch_num+1}: {epoch_time:.2f} seconds")
        print(f"Estimated remaining time: {format_time(remaining_time)} seconds")
        
        # Log epoch metrics to a file
        with open(log_file, "a") as f:
            f.write(f"Epoch {epoch_num+1}/{epochs}, Loss: {epoch_loss}, Accuracy: {epoch_acc}\n")
            f.write(f"Time for epoch {epoch_num+1}: {epoch_time:.2f} seconds\n")


        save_model(model, epoch_num, save_path)


In [12]:

class NERDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, max_len):
    """
    This is the constructor of the NERDataset
    Inputs:
    - x: a list of lists where each list contains the ids of the tokens
    - y: a list of lists where each list contains the label of each token in the sentence
    - pad: the id of the <PAD> token (to be used for padding all sentences and labels to have the same length)
    """
    # i guess x should be extended to have the same length as y
    self.x_tensor = x
    self.y_tensor = torch.tensor([seq + [0] * (max_len - len(seq)) for seq in y], dtype=torch.long)
    #################################################################################################################

  def __len__(self):
    """
    This function should return the length of the dataset (the number of sentences)
    """
    ###################### TODO: return the length of the dataset #############################

    return len(self.x_tensor)
  
    ###########################################################################################

  def __getitem__(self, idx):
    """
    This function returns a subset of the whole dataset
    """
    ###################### TODO: return a tuple of x and y ###################################
    return self.x_tensor[idx], self.y_tensor[idx]
    ##########################################################################################


In [13]:
def save_model(model, epoch, path="model_epoch_{}.pth"):
    torch.save(model.state_dict(), path.format(epoch))
    print(f"Model saved to {path.format(epoch)}")

In [14]:

class NERDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, max_len):
    """
    This is the constructor of the NERDataset
    Inputs:
    - x: a list of lists where each list contains the ids of the tokens
    - y: a list of lists where each list contains the label of each token in the sentence
    - pad: the id of the <PAD> token (to be used for padding all sentences and labels to have the same length)
    """
    # i guess x should be extended to have the same length as y
    self.x_tensor = x
    self.y_tensor = torch.tensor([seq + [0] * (max_len - len(seq)) for seq in y], dtype=torch.long)
    #################################################################################################################

  def __len__(self):
    """
    This function should return the length of the dataset (the number of sentences)
    """
    ###################### TODO: return the length of the dataset #############################

    return len(self.x_tensor)
  
    ###########################################################################################

  def __getitem__(self, idx):
    """
    This function returns a subset of the whole dataset
    """
    ###################### TODO: return a tuple of x and y ###################################
    return self.x_tensor[idx], self.y_tensor[idx]
    ##########################################################################################


In [15]:
def save_model(model, epoch, path="model_epoch_{}.pth"):
    torch.save(model.state_dict(), path.format(epoch))
    print(f"Model saved to {path.format(epoch)}")

In [16]:
train_SRC,output_labels = extract_sentences()
# train_SRC = train_SRC[:100000]
# output_labels = output_labels[:10000]
print("train_SRC read ",train_SRC[0])
print("output_labels read ",output_labels[0])
print("train_SRC length ",len(train_SRC))
print("output_labels length ",len(output_labels))
print("checkpoint 0")

ut_labels = read_unique_labels('./unique_labels.txt')

t_labels = {}
t_labels['0'] = 0
for i in range(len(ut_labels)):
    t_labels[ut_labels[i]] = i+1

train_SRC_size = len(train_SRC)

longest_sentence = 25
print("checkpoint 1")
#tag_indices = [[t_labels[tag] for tag in sentence_tags] for sentence_tags in output_labels]
for i in range(len(output_labels)):
    output_labels[i] = output_labels[i].split()
tag_indices = [[t_labels[tag] for tag in sentence_tags] for sentence_tags in output_labels]

model = NER(longest_sentence, hidden_size=512, embedding_dim=100,num_layers=1,
            dropout=0,bidirectional=False,batch_first=True,use_attention=False)
print("training src",train_SRC[0])
train_dataset = NERDataset(train_SRC, tag_indices, longest_sentence)
print("model created")
print("training starting")
print("checkpoint 2")
print("---------------------------------------------")
train(model, train_dataset, batch_size=1000, epochs=2, learning_rate=0.01,norm_clip=0.1)


100%|██████████| 2457/2457 [08:09<00:00,  5.02it/s]

Epochs: 2 | Train Loss: 311545.1875             | Train Accuracy: 0.0033599999733269215

Current Learning Rate: 0.000005
Epoch 2/2, Batch 2457/2457
Time for batch 2457: 0.10 seconds
Estimated remaining time: 0h 0m 0s seconds
Epochs: 2 | Train Loss: 752.5039953809284             | Train Accuracy: 0.006990815047174692

Current Learning Rate: 0.000005
Epoch 2/2, Loss: 752334.3221153846
Time for epoch 2: 489.92 seconds
Estimated remaining time: 0h 0m 0s seconds
Model saved to models_lstm/model_epoch_1.pth


In [84]:
def evaluate(model, test_dataset,batch_size=512, epochs=5):
    """
    This function takes a NER model and evaluates its performance (accuracy) on a test data
    Inputs:
    - model: a NER model
    - test_dataset: dataset of type NERDataset
    """    
    # (1) create the dataloader of the training set (make the shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

    epoch_times = []
    batch_times = []

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    else:
        print("CUDA is not available. Training on CPU ...")
    
    total_acc_test = 0
  
    # (2) disable gradients
    with torch.no_grad():

        for test_input, test_label in tqdm(test_dataloader):
            # (3) move the test input to the device
            test_label = test_label.to(device)

            # (4) move the test label to the device
            test_input = test_input.to(device)

            # (5) do the forward pass
            output = model(test_input)

            # accuracy calculation (just add the correct predicted items to total_acc_test)
            acc =  torch.sum(torch.argmax(output, dim=-1) == test_label).item()

            total_acc_test += acc
            
            # (6) calculate the over all accuracy
            
            total_acc_test /= (len(test_dataset) * test_dataset.y_tensor.size(1))

    print(f'\nTest Accuracy: {total_acc_test}')

In [ ]:
test_dataset = NERDataset()
evaluate(model, test_dataset)